# 파일 검색 및 검색 (File Search and Retrieval)

- **OpenAI 방식의 RAG 구현**

| 구분           | File Search                                              | Retrieval                                            |
| ------------ | -------------------------------------------------------- | ---------------------------------------------------- |
| **핵심 목적**    | 업로드된 파일을 빠르게 찾아서 모델이 참고하도록 돕는 도구                       | 임베딩 기반으로 문서 의미를 이해하고, 가장 관련된 내용을 뽑아주는 시스템          |
| **디자인 철학**   | 도구(Tool) — Responses API 내부에서 모델이 즉시 쓸 수 있도록 설계됨    | 서비스 레벨 API — 임베딩 + 검색을 조합해서 사용자가 직접 구축하는 검색 시스템 |
| **사용 대상**    | Responses API에서 `tools=[{"type": "file_search"}]` 형태로 지정 | 독립적인 `client.vector_stores.search()` 호출로 개발자가 직접 사용  |
| **내부 구조**    | 모델이 도구로 호출 → 백엔드에서 Vector Store 검색 수행                    | 사용자가 직접 Vector Store 생성, 파일 업로드, 검색, 필터링 수행          |
| **통합성**      | 모델 대화형 통합에 최적화 (자동 grounding, citations 등)               | 검색 로직을 직접 제어 가능 (chunk 크기, score threshold, 필터 등)    |
| **유스케이스 예시** | 이 PDF에서 환불 정책 찾아줘 → File Search로 모델이 자동 호출             | 내 FAQ 데이터에서 ‘환불’ 관련 문장 의미 검색 → Retrieval로 직접 쿼리    |

### 용도
단순 모델 대화에서 문서 참조만 원하면 `file_search`, 검색 시스템 구축이면 `retrieval` 사용

### 벡터 스토어 (Vector stores)

Vector store(벡터 스토어) 는 Retrieval API 와 File Search 도구에서 의미 기반 검색(Semantic Search) 을 가능하게 하는 핵심 컨테이너입니다. 
파일을 벡터 스토어에 추가하면, 해당 파일은 자동으로 청크 단위로 분할(chunked) 되고, 임베딩 (embedding) 이 생성되며, 인덱싱(indexing) 되어 검색 가능한 형태로 저장됩니다.  

- 구조 및 구성 요소
    - 벡터 스토어는 내부적으로 vector_store_file 객체들을 포함합니다.
    - 이 객체들은 실제로는 file 객체를 기반으로 만들어진 래퍼(wrapper) 타입입니다.
    - 즉, file → 실제 업로드된 원본 파일, vector_store.file → 해당 파일을 임베딩/인덱싱하여 벡터 스토어에 등록한 형태

| 객체 유형 (Object type)   | 설명 (Description)                                                                                                                                                  |
| --------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **file**              | Files API를 통해 업로드된 콘텐츠를 나타냅니다.                                                              |
| **vector_store**      | 검색 가능한 파일들을 담는 **컨테이너(Container)** 입니다.                                                                                                                           |
| **vector_store.file** | 벡터 스토어에 연결된 개별 파일을 나타내는 **래퍼(wrapper) 타입**입니다.<br>해당 파일은 이미 **청크 분할**, **임베딩**, **인덱싱** 처리가 완료된 상태입니다.<br>또한, **속성(attribute) 기반 필터링**을 위한 `attributes` 맵을 포함합니다. |

- 가격
| **저장 용량 (Storage)**      | **요금 (Cost)**      |
| ------------------------ | ------------------ |
| **총 1GB 이하 (모든 스토어 합산)** | **무료 (Free)**      |
| **1GB 초과분**              | **하루당 $0.10 / GB** |


## 파일 검색 (File Search)
모델이 응답을 생성하기 전에 관련 정보를 파일에서 검색할 수 있도록 합니다.
파일 검색은 Responses API에서 사용할 수 있는 도구입니다. 모델이 시맨틱 및 키워드 검색을 통해 이전에 업로드된 파일의 지식 기반에서 정보를 검색할 수 있도록 합니다. 벡터 저장소를 생성하고 파일을 업로드하면, 모델이 이러한 지식 기반 또는 벡터 저장소에 접근할 수 있도록 하여 모델의 내재된 지식을 강화할 수 있습니다.  

OpenAI에서 관리하는 호스팅 도구이므로 실행을 처리하기 위해 직접 코드를 구현할 필요가 없습니다. 모델이 이 도구를 사용하기로 결정하면 자동으로 도구를 호출하고 파일에서 정보를 검색하여 출력을 반환합니다.

Responses API로 파일 검색을 사용하려면 먼저 벡터 저장소에 지식 기반을 설정하고 파일을 업로드해야 합니다.  

지식 기반을 설정한 후에는 모델에서 사용할 수 있는 도구 목록에 file_search 도구를 포함할 수 있으며, 검색할 벡터 저장소 목록도 포함할 수 있습니다.

1. 벡터 저장소를 만들고 파일을 업로드합니다.

- `upload_and_poll()` - OpenAI의 Vector Store API 안에서 “파일 업로드(upload)” + “인덱싱 완료(poll)” 를 한 번에 처리하는 편의 함수

2. 검색어를 보내 관련 결과를 얻습니다.

In [ ]:
    # file_search 도구 설정
# 모델이 생성한 최종 텍스트 출력

- 응답에 검색 결과 포함
  
출력 텍스트에서 주석(파일 참조)을 볼 수 있지만, 파일 검색 호출은 기본적으로 검색 결과를 반환하지 않습니다.  
응답에 검색 결과를 포함하려면 응답을 생성할 때 include 매개변수를 사용할 수 있습니다.  

In [ ]:
    # file_search 도구 설정
    # include 옵션: 응답에 검색된 파일 결과(file_search_call.results)를 포함시킴
# 모델이 생성한 요약/최종 답변 출력

In [ ]:
# resp.output

### 다수 파일의 vector store 생성

In [ ]:
# "Financial Statements"라는 벡터 스토어 생성
# OpenAI에 업로드할 파일 준비
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.

In [ ]:
# 새 벡터 스토어 생성
# OpenAI에 업로드할 파일 준비
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.

# RAG (Retrieval Augmented Generation) - 검색 증강 생성


### 시맨틱 검색 (Semantic Search)

시맨틱 검색(Semantic Search) 은 벡터 임베딩(Vector Embeddings) 을 활용하여 의미적으로 연관된 결과를 찾아내는 검색 기법입니다.
이 방식의 핵심은, 단순히 키워드가 일치하지 않아도 의미가 유사한 결과를 찾을 수 있다는 것입니다.
즉, 기존의 키워드 기반 검색으로는 놓칠 수 있는 결과를 포착할 수 있습니다.

- 예시
검색 문장: “우리가 달에 간 게 언제였지?”  
가능한 결과 예시는 다음과 같습니다.
| 문장(텍스트)                    | 키워드 유사도 | 의미 유사도 |
| -------------------------- | ------- | ------ |
| 첫 번째 달 착륙은 1969년 7월에 일어났다. | 0%      | 65%    |
| 달에 간 첫 번째 사람은 닐 암스트롱이었다.   | 27%     | 43%    |
| 내가 달 케이크를 먹었을 때, 정말 맛있었다.  | 40%     | 28%    |


Retrieval API는 데이터에 대해 의미 기반 검색(Semantic Search) 을 수행할 수 있게 해줍니다.
이는 단순히 키워드가 일치하는 결과가 아니라, 의미적으로 유사한 정보를 찾아주는 기술입니다.  
예를 들어, “환불 정책이 뭐야?”라고 물으면 “반품 가능 기간은 30일입니다” 같은 문장을 찾아줍니다.  
이 검색 기능은 단독으로도 유용하지만, 모델과 결합하면 매우 강력한 지식 기반 응답 생성 시스템을 만들 수 있습니다.  

Retrieval은 벡터 스토어(Vector Store) 라는 데이터 인덱스를 기반으로 작동합니다.  
문서를 업로드하면 내부적으로 임베딩(embedding) 을 생성하고, 이를 통해 의미적으로 유사한 문서를 빠르게 찾아낼 수 있습니다.

### 벡터 스토어 생성/파일 업로드

In [ ]:
# 검색 쿼리를 전송하여 관련 결과를 가져옵니다.

In [ ]:
# file_search 결과에서 텍스트(content.text)만 추출하여 하나의 문자열로 합치는 코드

### 응답 합성 (Synthesizing responses)
검색 쿼리를 수행한 후에는 그 결과를 바탕으로 응답을 종합(synthesize) 하고 싶을 때가 있습니다.  
이를 위해 OpenAI의 모델을 활용할 수 있으며, 검색 결과(results) 와 원래의 질의(original query) 를 함께 제공하면,  
모델이 해당 자료에 근거한 사실 기반의 응답(grounded response) 을 생성해 줍니다.